# 加速：非同步爬蟲

* 了解非同步爬蟲加速原理與實作

## 作業目標

* 比較一下非同步爬蟲跟多線程爬蟲的差異是什麼？各自的優缺點為何？

# 非同步

In [1]:
URL = 'https://morvanzhou.github.io/'

In [4]:
import aiohttp, asyncio
import nest_asyncio
import time
nest_asyncio.apply()


async def job(session):
    response = await session.get(URL)                               #等待並切換
    return str(response.url)

async def main(loop):
    async with aiohttp.ClientSession() as session:                  #官網推薦建立Session的形式,也可以直接用request
        tasks = [loop.create_task(job(session)) for _ in range(2)]
        finished, unfinished = await asyncio.wait(tasks)            #收集完成的結果,會返回完成的和沒完成的,等全部都完成了才返回
        all_results = [r.result() for r in finished]                #獲取所有結果
        print(all_results)

t1 = time.time()
loop = asyncio.get_event_loop()
loop.run_until_complete(main(loop))
#loop.close()
print("Async total time:", time.time() - t1)

['https://morvanzhou.github.io/', 'https://morvanzhou.github.io/']
Async total time: 0.39005208015441895


# 多線程

In [6]:
import requests
import _thread

startTime = time.time()

for i in range(2):
    _thread.start_new_thread( requests.get, (URL, ) )
    print(URL)

finishTime = time.time()
print(finishTime - startTime)

Unclosed connection
client_connection: Connection<ConnectionKey(host='morvanzhou.github.io', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None)>


https://morvanzhou.github.io/
https://morvanzhou.github.io/
0.0006830692291259766


* 非同步是利用程式等待回應的時間，使用CPU處理額外步驟，是單線程，較不會出現額外等待時間。
* 多線程為平行運算，假如其中一線執行完，需要在等待其他線的完成，因此還是有額外等待的時間。
* 非同步使用CPU額外處裡較能利用額外的等待時間，但是多線程的平行處理會比較快，兩個合併使用為最佳。
* 多線程的缺點是會帶給系統上下文的切換造成額外負擔，執行多線程也會使共享變數出現鎖死的情況。